In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import math
import json
from PIL import Image
import os
from sklearn.metrics import confusion_matrix

In [ ]:
BATCH_SIZE = 32
VAL_BATCH_SIZE = 128
IMG_SIZE = (224, 224) # input size for mobilenet

In [ ]:
(ds_test_raw, ds_train_raw, ds_validation_raw), ds_info = tfds.load(
    'oxford_flowers102',
    split=['train', 'test', 'validation'], # Use test set as training set because it has the most examples
    with_info=True)

class_names = {}
for name in ds_info.features['label'].names:
  class_names[ds_info.features['label'].str2int(name)] = name

if not os.path.exists("labels.json"):
  with open("labels.json", "w") as json_file:
    json_file.write(json.dumps(class_names))

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/oxford_flowers102/2.1.1.incompleteYJ09ZB/oxford_flowers102-train.tfrecord


...

  0%|          | 0/6149 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/oxford_flowers102/2.1.1.incompleteYJ09ZB/oxford_flowers102-validation.tfrecord


  0%|          | 0/1020 [00:00<?, ? examples/s]

Dataset oxford_flowers102 downloaded and prepared to /root/tensorflow_datasets/oxford_flowers102/2.1.1. Subsequent calls will reuse this data.


In [ ]:
# Download sample images of each flower
sample_size = 6
saved_images = {}

def resize_and_crop(data):
  img_dims = tf.shape(data['image'])
  crop_size = tf.math.minimum(img_dims[0], img_dims[1])
  image = tf.image.crop_to_bounding_box(
      data['image'], 
      offset_height=(img_dims[0] - crop_size) // 2, 
      offset_width=(img_dims[1] - crop_size) // 2,
      target_height=crop_size,
      target_width=crop_size)
  image = tf.image.resize(image, IMG_SIZE)
  return image

def save_sample_images():
  samples_dir = "samples/"
  if not os.path.exists(samples_dir):
    os.mkdir(samples_dir)

    for example in ds_train_raw:
      label = example['label'].numpy()
      path = f"samples/{class_names[label]}"
      if label in saved_images:
        if saved_images[label] >= 6:
          continue
        else:
          saved_images[label] += 1
      else:
        saved_images[label] = 1
        os.mkdir(path)

      arr = np.uint8(resize_and_crop(example).numpy())
      im = Image.fromarray(arr)
      im.save(f"{path}/{saved_images[label]}.jpg")

save_sample_images()

In [ ]:
def preprocess_img(data):
  # Crop image to largest center square, resize to fit feature net input
  image = resize_and_crop(data)
  image = tf.keras.applications.mobilenet.preprocess_input(image)
  return image, data['label']

In [ ]:
# Create train and test datasets

ds_train = ds_train_raw.map(preprocess_img,
  num_parallel_calls=tf.data.AUTOTUNE).cache().shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

ds_validation = ds_validation_raw.map(preprocess_img, 
  num_parallel_calls=tf.data.AUTOTUNE).batch(VAL_BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)

ds_test = ds_test_raw.map(preprocess_img, 
  num_parallel_calls=tf.data.AUTOTUNE).batch(ds_info.splits['train'].num_examples).cache().prefetch(tf.data.AUTOTUNE)

In [ ]:
feature_model = tf.keras.applications.MobileNet(include_top=False,
                                              weights='imagenet',
                                              input_shape=(224, 224, 3))
feature_model.trainable = False

model = tf.keras.models.Sequential(
  feature_model.layers + [
    tf.keras.layers.MaxPooling2D(pool_size=(7, 7)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(ds_info.features['label'].num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0         
_________________________________________________________________
conv_pw_1 (Conv2D)           (None, 112, 112, 64)      2

In [ ]:
checkpoint_filepath = './tmp/checkpoint'
checkpoint = tf.keras.callbacks.ModelCheckpoint(monitor="val_accuracy",
                                                mode="max",
                                                filepath=checkpoint_filepath,
                                                save_best_only=True)

In [ ]:
history = model.fit(
    ds_train,
    epochs=50,
    validation_data=ds_validation,
    callbacks=[checkpoint]
)
model.load_weights(checkpoint_filepath)
model.save("model.h5")

Epoch 1/50
193/193 [==============================] - 31s 128ms/step - loss: 4.0449 - accuracy: 0.3833 - val_loss: 2.2918 - val_accuracy: 0.5490
INFO:tensorflow:Assets written to: ./tmp/checkpoint/assets


INFO:tensorflow:Assets written to: ./tmp/checkpoint/assets


Epoch 2/50
193/193 [==============================] - 17s 89ms/step - loss: 0.7937 - accuracy: 0.7962 - val_loss: 1.2702 - val_accuracy: 0.7265
INFO:tensorflow:Assets written to: ./tmp/checkpoint/assets


INFO:tensorflow:Assets written to: ./tmp/checkpoint/assets


Epoch 3/50
193/193 [==============================] - 17s 89ms/step - loss: 0.3137 - accuracy: 0.9106 - val_loss: 1.0261 - val_accuracy: 0.7667
INFO:tensorflow:Assets written to: ./tmp/checkpoint/assets


...

Epoch 41/50
193/193 [==============================] - 17s 88ms/step - loss: 1.0229e-04 - accuracy: 1.0000 - val_loss: 0.6295 - val_accuracy: 0.9069
Epoch 42/50
193/193 [==============================] - 17s 88ms/step - loss: 7.3419e-05 - accuracy: 1.0000 - val_loss: 0.6239 - val_accuracy: 0.9098
INFO:tensorflow:Assets written to: ./tmp/checkpoint/assets


INFO:tensorflow:Assets written to: ./tmp/checkpoint/assets


Epoch 43/50
193/193 [==============================] - 17s 88ms/step - loss: 6.5321e-05 - accuracy: 1.0000 - val_loss: 0.6195 - val_accuracy: 0.9108
INFO:tensorflow:Assets written to: ./tmp/checkpoint/assets


INFO:tensorflow:Assets written to: ./tmp/checkpoint/assets


Epoch 44/50
193/193 [==============================] - 17s 88ms/step - loss: 5.9804e-05 - accuracy: 1.0000 - val_loss: 0.6165 - val_accuracy: 0.9088
Epoch 45/50
193/193 [==============================] - 17s 88ms/step - loss: 5.5357e-05 - accuracy: 1.0000 - val_loss: 0.6133 - val_accuracy: 0.9098
Epoch 46/50
193/193 [==============================] - 17s 88ms/step - loss: 5.1318e-05 - accuracy: 1.0000 - val_loss: 0.6103 - val_accuracy: 0.9088
Epoch 47/50
193/193 [==============================] - 17s 88ms/step - loss: 4.8310e-05 - accuracy: 1.0000 - val_loss: 0.6077 - val_accuracy: 0.9088
Epoch 48/50
193/193 [==============================] - 17s 88ms/step - loss: 4.5216e-05 - accuracy: 1.0000 - val_loss: 0.6051 - val_accuracy: 0.9098
Epoch 49/50
193/193 [==============================] - 17s 87ms/step - loss: 4.2526e-05 - accuracy: 1.0000 - val_loss: 0.6025 - val_accuracy: 0.9098
Epoch 50/50
193/193 [==============================] - 17s 87ms/step - loss: 4.0196e-05 - accuracy: 1.0000

In [ ]:
# Test model

model = tf.keras.models.load_model("model (1).h5")
model.evaluate(ds_test)

1/1 [==============================] - 40s 40s/step - loss: 0.8108 - accuracy: 0.8863


[0.8107962608337402, 0.886274516582489]

In [ ]:
# Calculate normalized confusion matrix
# Diagonals of the matrix are the recall for each class (TP / (TP + FN))
# Normalize each row; TP + FN = 1

test_data = list(ds_test)[0]
y_pred = tf.argmax(model.predict(test_data[0]), 1).numpy()
y_true = test_data[1].numpy()

conf_matrix = confusion_matrix(y_true, y_pred)
sum_of_rows = conf_matrix.sum(axis=1)
conf_matrix = conf_matrix / sum_of_rows[:, np.newaxis]
conf_matrix

array([[0.6, 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 1. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0.2, ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0.9, 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0.8, 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 1. ]])

In [ ]:
# Calculate similarity matrix using manhattan distance

conf_matrix_copy = np.copy(conf_matrix)
dist_matrix = np.zeros(conf_matrix.shape)

while conf_matrix_copy.shape[0]:
  eval_row = conf_matrix_copy[0]
  dist_vector = np.sum(np.abs(conf_matrix_copy - eval_row), axis=1)
  current_row_index = conf_matrix.shape[0] - conf_matrix_copy.shape[0]
  dist_matrix[current_row_index] = np.pad(
      dist_vector,
      pad_width=(current_row_index, 0),
      mode='constant')

  conf_matrix_copy = np.delete(conf_matrix_copy, (0), axis=0)

dist_matrix

array([[0., 2., 2., ..., 2., 2., 2.],
       [0., 0., 2., ..., 2., 2., 2.],
       [0., 0., 0., ..., 2., 2., 2.],
       ...,
       [0., 0., 0., ..., 0., 2., 2.],
       [0., 0., 0., ..., 0., 0., 2.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
# Find similar class pairs (pairs with a distance < 2)
pairs = []
for i in range(dist_matrix.shape[0]):
  for j in range(i + 1, dist_matrix.shape[1]):
    if dist_matrix[i][j] < 2:
      pairs.append(((class_names[i], class_names[j]), dist_matrix[i][j]))

pairs = sorted(pairs, key=lambda x: x[1])

In [ ]:
pairs[:10]

[(('spear thistle', 'artichoke'), 1.4),
 (('globe thistle', 'common dandelion'), 1.5999999999999999),
 (('garden phlox', 'japanese anemone'), 1.5999999999999999),
 (('primula', 'japanese anemone'), 1.5999999999999999),
 (('canterbury bells', 'moon orchid'), 1.6),
 (('canterbury bells', 'giant white arum lily'), 1.6),
 (('canterbury bells', 'desert-rose'), 1.6),
 (('canterbury bells', 'cyclamen'), 1.6),
 (('moon orchid', 'giant white arum lily'), 1.6),
 (('globe thistle', 'bee balm'), 1.6)]

In [ ]:
os.mkdir("models")
os.mkdir("models/model")

MODEL_DIR = "models/model"
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

export_path = models/model/1

INFO:tensorflow:Assets written to: models/model/1/assets


INFO:tensorflow:Assets written to: models/model/1/assets
